In [1]:
import pandas as pd
import numpy as np

In [2]:
#We have two resampled and therefore regular datasets, now we need to make our time series into a supervised problem.
#First I need to change navstat into a categorical feature:


# Define categories based on ranges or discrete values
pretty_20m = pd.read_csv("../Project materials(1)/resampled_data_20min.csv")

pretty_h = pd.read_csv("../Project materials(1)/resampled_data_h.csv")

navstat_unique = pretty_h["navstat"].unique()

pretty_20m["navstat"] = pd.Categorical(pretty_20m["navstat"], categories=navstat_unique, ordered=True)

pretty_h["navstat"] = pd.Categorical(pretty_h["navstat"], categories=navstat_unique, ordered=True)


# Let's make dummys 

pretty_h = pd.get_dummies(pretty_h, columns=["navstat"], drop_first=True)
pretty_20m = pd.get_dummies(pretty_20m, columns = ["navstat"], drop_first=True)

pretty_20m.set_index("time", inplace=True)
pretty_h.set_index("time", inplace=True)

In [3]:
#Make time series into supervised problem
def make_supervised(df, forecast_columns, sorting_column, input_window=1, output_window=1):
    """
    Converts a multivariate time series dataframe into a supervised learning problem.
    
    Parameters:
    df (pd.DataFrame): The original dataframe with time series data.
    forecast_columns (list): A list of column names to forecast.
    input_window (int): The number of past observations to use as features.
    output_window (int): The number of steps to forecast into the future.
    
    Returns:
    pd.DataFrame: A new dataframe with supervised learning format.
    """
    # Create a list to hold the transformed value
    df_holder = []
    
    #Put in a for loop here where you iterate over all IDs, to make sure things get correct
    unique_sorts = df[sorting_column].unique()
    
    #Iterate through all IDs
    for sorts in unique_sorts:
        df_supervised = pd.DataFrame()
        sort_df = df[df[sorting_column] == sorts]

        #Iterate through all columns for input features
        for col in sort_df.columns: 
            for i in range(input_window, 0, -1):
                df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
            df_supervised[f"{col}_t"] = sort_df[col]
            

    # Create columns for forecast (target) with forward shift
        for col in forecast_columns:
            for j in range(output_window, 0,-1):
                df_supervised[f"{col}_t+{j}"] = sort_df[col].shift(-j)
        
        df_holder.append(df_supervised)
    

    
    df_new = pd.DataFrame()
    
    for chunk in df_holder:
        df_new = pd.concat([df_new, chunk])
    # Remove rows with NaN values caused by the shifting process
    df_new.dropna(inplace=True)
    
    return df_new

supervised_h = make_supervised(pretty_h, ["latitude", "longitude"],"vesselId" , 3, 2)

supervised_h.columns

/tmp/ipykernel_34912/3647064159.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
/tmp/ipykernel_34912/3647064159.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_supervised[f"{col}_t-{i}"] = sort_df[col].shift(i)
/tmp/ipykernel_34912/3647064159.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. 

Index(['cog_t-3', 'cog_t-2', 'cog_t-1', 'cog_t', 'sog_t-3', 'sog_t-2',
       'sog_t-1', 'sog_t', 'rot_t-3', 'rot_t-2',
       ...
       'navstat_13.0_t-1', 'navstat_13.0_t', 'navstat_9.0_t-3',
       'navstat_9.0_t-2', 'navstat_9.0_t-1', 'navstat_9.0_t', 'latitude_t+2',
       'latitude_t+1', 'longitude_t+2', 'longitude_t+1'],
      dtype='object', length=112)

In [ ]:
#Sorting by time
supervised_h = supervised_h.sort_index(ascending = True)

#Train, test and validation splits, this can be tuned.


In [ ]:
print(supervised_h.columns)

def train_test_split(df, perc1, perc2, output_window):
    y_list = []
    for j in range(output_window):
        y_list.append(f"{"longitude"}_t-{j+1}")
        y_list.append(f"{"latitude"}_t-{j+1}")
    ys = df[y_list]
    Xs = df.drop(columns = y_list)

    X_train = Xs[:]



ys = supervised_h["longitude_"]

Index(['cog_t-3', 'cog_t-2', 'cog_t-1', 'cog_t', 'sog_t-3', 'sog_t-2',
       'sog_t-1', 'sog_t', 'rot_t-3', 'rot_t-2',
       ...
       'navstat_13.0_t-1', 'navstat_13.0_t', 'navstat_9.0_t-3',
       'navstat_9.0_t-2', 'navstat_9.0_t-1', 'navstat_9.0_t', 'latitude_t+2',
       'latitude_t+1', 'longitude_t+2', 'longitude_t+1'],
      dtype='object', length=112)
